La función de Rosenbrok es una función que se usa bastante para poder probar algoritmos de optimización. Lo que tienen que hacer es :

- Implementar la función de costo para poderla optimizar.
- Usar el algoritmo de Newton para intentar optimizarla.
- Usar el algoritmo de búsqueda lineal para optimizarla.
- Extra(opcional): Hagan la gráfica de la función para entender por qué es difícil.


## Función de Rosenbrock
$ f(x,y) = (a-x)^2 + b(y-x^2)^2$

Nota: en mi caso voy a usar los parametros que sugiere Wikipedia, es decir, a=1 y b=100 para definir la función de costos
Nota: tiene un mínimo global en $(a,a^2)$

In [78]:
import numpy as np
x0 = [x,y]
def funcion_costos(x0):
    return (1-x0[0])** 2 + (100*(x0[1]-x0[0]**2))**2

In [79]:
x = np.linspace(-2,2,100)
y = np.linspace(-0.5,3,100)
x0 = [x,y]
funcion_costos(x0)

array([2.02509000e+05, 1.85309977e+05, 1.69142532e+05, 1.53969040e+05,
       1.39752514e+05, 1.26456605e+05, 1.14045607e+05, 1.02484451e+05,
       9.17387079e+04, 8.17745892e+04, 7.25589454e+04, 6.40592667e+04,
       5.62436829e+04, 4.90809632e+04, 4.25405168e+04, 3.65923923e+04,
       3.12072778e+04, 2.63565010e+04, 2.20120296e+04, 1.81464703e+04,
       1.47330700e+04, 1.17457146e+04, 9.15893019e+03, 6.94788204e+03,
       5.08837521e+03, 3.55685432e+03, 2.33040360e+03, 1.38674688e+03,
       7.04247582e+02, 2.61908756e+02, 3.93730350e+01, 1.69226617e+01,
       1.75479480e+02, 4.96604938e+02, 9.62500086e+02, 1.55600558e+03,
       2.26060167e+03, 3.06040821e+03, 3.94018468e+03, 4.88533014e+03,
       5.88188324e+03, 6.91652227e+03, 7.97656510e+03, 9.04996920e+03,
       1.01253317e+04, 1.11918891e+04, 1.22395180e+04, 1.32587340e+04,
       1.42406927e+04, 1.51771892e+04, 1.60606582e+04, 1.68841740e+04,
       1.76414505e+04, 1.83268412e+04, 1.89353392e+04, 1.94625772e+04,
      

In [132]:
#probamos el mínimo global
x0 = [1,1]
funcion_costos(x0)

0

In [133]:
#codigo de la clase: 

def Grad(f, x0, h=1e-6, i=-1):
    """
    Función que calcula el Grad de una función en un punto
    """
    n = len(x0)
    if i in range(n):
        z = np.zeros(n)
        z[i] = h/2
        Grad = (f(x0 + z) - f(x0 - z))/h
    else:
        Grad=np.zeros(n)
        for j in range(n):
            z = np.zeros(n)
            z[j] = h/2
            Grad[j]= (f(x0 + z) - f(x0 - z))/h
    return Grad


def Hess(f, x0, h=1e-4, method = "basic"):
    """
    Función que calcula la Hessiana  de una función en un punto. 
    f: función sobre la cual queremos calcular la hessiana.
    x0: Punto sobre el cual queremos hacer el cálculo
    h: nivel de precisión para hacer el cálculo
    method: Método por el cual se quiere hacer puede ser: 'basic', 'grad', 'centered', 'gradCentered'
    """
    n = len(x0)
    Hess = np.matrix(np.zeros((n,n)))
    for i in range(n):
        for j in range(n):
            z_i = np.zeros(n)
            z_i[i] = h
            z_j = np.zeros(n)
            z_j[j] = h
            if method == "basic":
                Hess[i,j] = ( f(x0 + z_j +z_i) - f(x0 + z_i ) - f(x0+z_j) +f(x0)) / (h**2)
            elif method == "grad":
                Hess[i,j] = (Grad(f,x0+z_j,h,i) - Grad(f,x0,h,i) + \
                             Grad(f,x0+z_i,h,j) - Grad(f,x0,h,j))/(2*h)
            elif method == "centered":
                if i==j:
                    Hess[i,j] = (-f(x0+2*z_i) + 16*f(x0+z_i) - 30*f(x0)+\
                                 16*f(x0-z_i) - f(x0-2*z_i))  / (12*h**2)
                else :
                    Hess[i,j] = (f(x0+z_i+z_j) - f(x0 + z_i - z_j) - \
                                 f(x0 - z_i + z_j) + f(x0-z_i-z_j))/(4*h**2)
            elif method == "gradCentered":
                    Hess[i,j] = (Grad(f,x0+z_j,h)[i] - Grad(f, x0-z_j,h)[i] + \
                                 Grad(f,x0+z_i,h)[j] - Grad(f,x0-z_i,h)[j])/(4*h)
    return Hess

In [134]:
def f_o_c(f,x, tol=1e-12):
    """
    Función que calcula las condiciones de primer orden
    """
    grad = np.array(Grad(f,x))
    if np.dot(grad, grad) < tol:
        return True
    else :
        return False

def s_o_c(f, x0, tol=1e-15):
    """
    Inserten aqui código para condiciones de segundo orden 
    """
    hess = Hess(f, x0, tol)
    print(np.linalg.eigvals(hess))
    if np.all(np.linalg.eigvals(hess) > tol) :
        return True
    else :
        return False


def is_min(f, x0, tol=1e-25) :
    """
    Dado una función, la función is_min nos va a regresar True si es un mínimo, 
    False si no podemos garantizar que es un mínimo
    """
    if f_o_c(f, x0, tol) and s_o_c(f, x0, tol) :
        return True
    else :
        return False

In [82]:
def genera_alpha(f, x0, pk, c1=1e-4, tol=1e-5):
    """
    Backtracking LS i.e. Algoritmo que encuentra una alpha que cumpla condiciones de wolfe. 
    """
    alpha, rho, c = 1, 4/5, c1
    while f(x0 + alpha*pk)>f(x0) + c*alpha*np.dot(Grad(f, x0),pk):
        alpha*=rho
    return alpha

In [84]:
def busqueda_lineal_newton(f,x0):
    xk = x0
    while not is_min(f,xk,tol=1e-30):
        #pk= (-1)*Grad(f,xk)
        pk = np.linalg.solve(Hess(f,xk),Grad(f,xk)) # direccion de Newton
        xk = xk + pk
    return xk

In [ ]:
def busqueda_lineal_newton(f,x0):
    xk = x0
    while not is_min(f,xk,tol=1e-30):
        #pk= (-1)*Grad(f,xk)
        pk = np.linalg.solve(Hess(f,xk),Grad(f,xk)) # direccion de Newton
        #pk = np.dot((-1)*np.linalg.inv(Hess(f,xk)),Grad(f,xk).T)
        alpha_k = genera_alpha(f,x0,pk)
        xk = xk + (alpha_k*pk)
    return xk

In [85]:
busqueda_lineal_newton(funcion_costos,[0,0])

KeyboardInterrupt: 

In [131]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt


fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

#a = 1, b= 100
def funcion_costos(x0):
    return (1-x0[0])** 2 + (100*(x0[1]-x0[0]**2))**2

x = np.linspace(-2,2,100)
y = np.linspace(-0.5,3,100)

X, Y = np.meshgrid(x, y)
x0 = [X,Y] #lista
Z = funcion_costos(x0)

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('f')

ax.plot_surface(X, Y, Z, cmap='coolwarm')


<img src="Figure_1.png" alt="Girl in a jacket" width="500" height="600">